- As project 2, I made functions at .py file, and then ,plot results in .ipynb file. 
- I condider Repeated First Price Auction(hereinafter called repeated FPA), not second price auction.
- In addition, I consider battle of sexes for supplementary.
- Players in Repeated FPA must have algorithm whose regret coverges to 0. 

- Full Information: Each player knows their own value (v) which is fixed across all rounds
- Full Feedback: After each round, each player observes all bids from all players, not just their own outcome

This setting allows players to use opponent's bid history directly in their learning algorithms.

# Project 3

# Part 1

## repeated FPA

### Algorithms and Parameters

1. 1_empirical: Empirical algorithm - always maximizes current round expected utility based on past opponent's bid data. 
2. 2_ew: EW - Exponential Weight with learning_rate
3. 3_FTL: Follow The Leader algorithm - selects arm with highest cumulative payoff
4. 4_uniform_guessing: Uniform Guessing algorithm - randomly selects bids uniformly
5. 5_exploitation: Exploitation algorithm - waits and exploits when opponent bids low

- n_rounds: 1000 - number of rounds per simulation
- k: 100 - number of discrete arms (discretization level)
- n_mc: 100 - number of Monte Carlo simulation runs
- h: scaling parameter (default: value) - used in Exponential Weight algorithms
- value (v): 10.0 - player's value for the item (default)
- learning_rate: sqrt(log(k) / n) - learning rate for Exponential Weight algorithms (default for flexible)
  Optimal learning rate: epsilon = sqrt(log(k) / T)
- observation_rounds: 5 - number of observation rounds for exploitation algorithm (default)

In [1]:
import numpy as np
import sys, importlib
from pathlib import Path
sys.path.insert(0, str(Path('algorithm').resolve()))

empirical, ew, exploitation = [importlib.import_module(m) for m in ['1_Empirical', '2_ew', '5_exploitation']]
# Import new algorithms: FTL and Uniform Guessing
ftl = importlib.import_module('3_FTL')
uniform_guessing = importlib.import_module('4_uniform_guessing')

import repeated_FPA
from repeated_FPA import run_repeated_fpa, plot_results

In [2]:
# Parameters
n_rounds = 10000
k = 100
n_mc = 100

### Experiment

In [ ]:
v1, v2 = 1.0, 1.0

player1 = (ew.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': None}) # None = default (sqrt(log(k) / n)), or specify a value like 0.1
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': None})  
results_ew_vs_ew = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_ew_vs_ew, title="EW vs EW", show_ne_distance=True)

MC iteration 10/100 completed


In [ ]:
v1, v2 = 1.0, 1.0

player1 = (ftl.ftl_algorithm, v1, {'k': k, 'h': v1})
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': None})
results_FTL_vs_ew = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_FTL_vs_ew, title="FTL vs EW")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Completed: FTL vs EW
Plot 1 saved to: ../figures/ftl_vs_ew_bid_evolution.png
Plot 2 saved to: ../figures/ftl_vs_ew_regret.png
Plot 4 saved to: ../figures/ftl_vs_ew_utility_distribution.png
Plot 5 saved to: ../figures/ftl_vs_ew_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 4.46 ± 1.08
  Mean Utility: 355.88 ± 21.25
  Mean Win Rate: 0.507 ± 0.014

Player 2:
  Mean Regret: 78.44 ± 5.69
  Mean Utility: 304.40 ± 29.47
  Mean Win Rate: 0.493 ± 0.014
Summary statistics saved to: ../data/ftl_vs_ew_summary.csv
Detailed results saved to: ../data/ftl_vs_ew_detailed.csv
Regret history saved to: ../data/ftl_vs_ew_regret_history.csv
Bid history saved to: ../data/ftl_vs_ew

In [ ]:
v1, v2 = 1.0, 1.0

player1 = (uniform_guessing.uniform_guessing_algorithm, v1, {'k': k})
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': None})
results_uniform_vs_ew = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_uniform_vs_ew, title="Uniform guessing vs EW")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Completed: Uniform guessing vs EW
Plot 1 saved to: ../figures/uniform_guessing_vs_ew_bid_evolution.png
Plot 2 saved to: ../figures/uniform_guessing_vs_ew_regret.png
Plot 4 saved to: ../figures/uniform_guessing_vs_ew_utility_distribution.png
Plot 5 saved to: ../figures/uniform_guessing_vs_ew_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 2514.84 ± 56.25
  Mean Utility: 1278.34 ± 30.43
  Mean Win Rate: 0.504 ± 0.007

Player 2:
  Mean Regret: 51.24 ± 9.62
  Mean Utility: 2449.21 ± 27.25
  Mean Win Rate: 0.496 ± 0.007
Summary statistics saved to: ../data/uniform_guessing_vs_ew_summary.csv
Detailed results saved to: ../data/uniform_guessing_vs_ew_detailed.csv
Regr

In [ ]:
v1, v2 = 1.0, 1.0

import numpy as np
optimal_lr = np.sqrt(np.log(k) / n_rounds)
lr_3x = 3 * optimal_lr

player1 = (ew.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': None})
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': lr_3x})

results_lr_comparison = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_lr_comparison, title="Optimal LR vs 3x Optimal LR")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Completed: Optimal LR vs 3x Optimal LR
Plot 1 saved to: ../figures/optimal_lr_vs_3x_optimal_lr_bid_evolution.png
Plot 2 saved to: ../figures/optimal_lr_vs_3x_optimal_lr_regret.png
Plot 4 saved to: ../figures/optimal_lr_vs_3x_optimal_lr_utility_distribution.png
Plot 5 saved to: ../figures/optimal_lr_vs_3x_optimal_lr_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 81.07 ± 5.15
  Mean Utility: 537.02 ± 16.52
  Mean Win Rate: 0.505 ± 0.003

Player 2:
  Mean Regret: 69.00 ± 6.12
  Mean Utility: 532.35 ± 14.66
  Mean Win Rate: 0.495 ± 0.003
Summary statistics saved to: ../data/optimal_lr_vs_3x_optimal_lr_summary.csv
Detailed results saved to: ../data/optimal_lr_vs_3

In [ ]:
v1, v2 = 1.0, 1.0

player1 = (empirical.empirical_algorithm, v1, {'k': k, 'h': v1})
player2 = (ew.flexible_algorithm, v2, {'k': k, 'h': v2, 'learning_rate': None})
results_empirical_vs_ew = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_empirical_vs_ew, title="Empirical vs EW", show_ne_distance=True)

In [ ]:
from value_generate import generate_value
player1 = (ew.flexible_algorithm, lambda: generate_value('uniform', low=0.0, high=1.0), {'k': k, 'learning_rate': None}) 
player2 = (ew.flexible_algorithm, lambda: generate_value('uniform', low=0.0, high=1.0), {'k': k, 'learning_rate': None}) 
results_ew_vs_ew_from_uniform_distribution = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
plot_results(results_ew_vs_ew_from_uniform_distribution, title="EW vs EW from uniform distribution")

/Users/harashimakoshi/CS332/332Project3/code/algorithm/2_ew.py:147: RuntimeWarning: overflow encountered in power
  powers = (1 + learning_rate) ** (valid_payoffs / h)


Completed: EW vs EW from uniform distribution
Plot 1 saved to: ../figures/ew_vs_ew_from_uniform_distribution_bid_evolution.png
Plot 2 saved to: ../figures/ew_vs_ew_from_uniform_distribution_regret.png
Plot 4 saved to: ../figures/ew_vs_ew_from_uniform_distribution_utility_distribution.png
Plot 5 saved to: ../figures/ew_vs_ew_from_uniform_distribution_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 42.19 ± 0.00
  Mean Utility: 971.94 ± 0.00
  Mean Win Rate: 0.492 ± 0.000

Player 2:
  Mean Regret: 29.05 ± 0.00
  Mean Utility: 967.82 ± 0.00
  Mean Win Rate: 0.508 ± 0.000
Summary statistics saved to: ../data/ew_vs_ew_from_uniform_distribution_summary.csv
Detailed results saved to: ../data/ew_vs_ew_from_uniform_distribution_detailed.csv
Regret history saved to: ../data/ew_vs_ew_from_uniform_distribution_regret_history.csv
Bid history saved to: ../data/ew_vs_ew_from_uniform_distribution_bid_history.csv


## battle of sexes

### Algorithms and Parameters

In [ ]:
# Import bimatrix game functions
import sys
sys.path.insert(0, str(Path('.').resolve()))
import repeated_bimatrix
from repeated_bimatrix import run_repeated_bimatrix, plot_ne_convergence
from algorithm import bimatrix_wrapper

payoff_matrix = np.array([
    [[2, 4], [0, 0]],  # Player 1 plays A
    [[0, 0], [4, 2]]   # Player 1 plays B
])

print("Payoff Matrix:")
print("        Action A    Action B")
print(f"Action A  ({payoff_matrix[0,0,0]},{payoff_matrix[0,0,1]})      ({payoff_matrix[0,1,0]},{payoff_matrix[0,1,1]})")
print(f"Action B  ({payoff_matrix[1,0,0]},{payoff_matrix[1,0,1]})      ({payoff_matrix[1,1,0]},{payoff_matrix[1,1,1]})")
print("\nPure Nash Equilibria:")
ne_pure = repeated_bimatrix.find_pure_nash_equilibria(payoff_matrix)
for ne in ne_pure:
    print(f"  ({chr(65+ne[0])}, {chr(65+ne[1])})")

In [ ]:
# Parameters for Battle of Sexes game
n_rounds = 10000
n_mc = 100

### Experiments

In [ ]:
# Test 1: Flexible vs Flexible
player1_coord = (bimatrix_wrapper.flexible_bimatrix, {'learning_rate': None, 'h': 3.0})
player2_coord = (bimatrix_wrapper.flexible_bimatrix, {'learning_rate': None, 'h': 3.0})

results_flexible_vs_flexible = run_repeated_bimatrix(
    player1_coord, player2_coord, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_flexible_vs_flexible, title="Flexible vs Flexible (Battle of Sexes)")

In [ ]:
# Test 2: Flexible vs Empirical (robustness check)
player1_coord2 = (bimatrix_wrapper.flexible_bimatrix, {'learning_rate': None, 'h': 3.0})
player2_coord2 = (bimatrix_wrapper.empirical_bimatrix, {})

results_flexible_vs_empirical = run_repeated_bimatrix(
    player1_coord2, player2_coord2, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_flexible_vs_empirical, title="Flexible vs Empirical (Battle of Sexes)")

In [ ]:
# Test 3: FTL vs Flexible (robustness check)
player1_coord3 = (bimatrix_wrapper.ftl_bimatrix, {})
player2_coord3 = (bimatrix_wrapper.flexible_bimatrix, {'learning_rate': None, 'h': 4.0})

results_ftl_vs_flexible = run_repeated_bimatrix(
    player1_coord3, player2_coord3, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_ftl_vs_flexible, title="FTL vs Flexible (Battle of Sexes)")

In [ ]:
# Robustness: Test with different initial conditions
from typing import List, Tuple

def flexible_bimatrix_with_init(player_id: int, round_num: int, history: List[Tuple[int, float, int]],
                                env_state: dict, payoff_matrix, initial_action=None) -> int:
    """Flexible algorithm with optional initial action."""
    if round_num == 0 and initial_action is not None:
        return initial_action
    return bimatrix_wrapper.flexible_bimatrix(player_id, round_num, history, env_state, payoff_matrix)

# Test with different initial conditions
print("=" * 60)
print("Robustness Test: Different Initial Conditions")
print("=" * 60)

# Case 1: Both start with action A
player1_init_A = (lambda pid, rn, h, es, pm: flexible_bimatrix_with_init(pid, rn, h, es, pm, initial_action=0),
                  {'learning_rate': None, 'h': 3.0})
player2_init_A = (lambda pid, rn, h, es, pm: flexible_bimatrix_with_init(pid, rn, h, es, pm, initial_action=0),
                  {'learning_rate': None, 'h': 3.0})

results_init_AA = run_repeated_bimatrix(
    player1_init_A, player2_init_A, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_init_AA, title="Flexible vs Flexible (Initial: A, A)")

# Case 2: Both start with action B
player1_init_B = (lambda pid, rn, h, es, pm: flexible_bimatrix_with_init(pid, rn, h, es, pm, initial_action=1),
                  {'learning_rate': None, 'h': 3.0})
player2_init_B = (lambda pid, rn, h, es, pm: flexible_bimatrix_with_init(pid, rn, h, es, pm, initial_action=1),
                  {'learning_rate': None, 'h': 3.0})

results_init_BB = run_repeated_bimatrix(
    player1_init_B, player2_init_B, payoff_matrix, n_rounds, n_mc
)
plot_ne_convergence(results_init_BB, title="Flexible vs Flexible (Initial: B, B)")

# Part 2
- We implement exploitation strategy for repeated FPA
- Exploitation strategy vs 2_ew (Exponential Weight algorithm)

### Algorithms and Parameters

In [ ]:
import numpy as np
import sys, importlib
from pathlib import Path
sys.path.insert(0, str(Path('algorithm').resolve()))

empirical, ew, exploitation = [importlib.import_module(m) for m in ['1_Empirical', '2_ew', '5_exploitation']]
ftl = importlib.import_module('3_FTL')
uniform_guessing = importlib.import_module('4_uniform_guessing')

import repeated_FPA
from repeated_FPA import run_repeated_fpa, plot_results

In [ ]:
# Parameters
n_rounds = 10000
k = 100
n_mc = 100

### Experiments

In [15]:
v1, v2 = 0.9, 0.3
player1 = (ew.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': None})  
player2 = (exploitation.exploitation_algorithm, v2, {'k': k, 'h': v2, 'observation_rounds': 5})
results_ew_vs_exploitation = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
print("Completed: EW vs Exploitation")
plot_results(results_ew_vs_exploitation, title="EW vs Exploitation")

MC iteration 10/100 completed
MC iteration 20/100 completed
MC iteration 30/100 completed
MC iteration 40/100 completed
MC iteration 50/100 completed
MC iteration 60/100 completed
MC iteration 70/100 completed
MC iteration 80/100 completed
MC iteration 90/100 completed
MC iteration 100/100 completed
Completed: EW vs Exploitation
Plot 1 saved to: ../figures/ew_vs_exploitation_bid_evolution.png
Plot 2 saved to: ../figures/ew_vs_exploitation_regret.png
Plot 4 saved to: ../figures/ew_vs_exploitation_utility_distribution.png
Plot 5 saved to: ../figures/ew_vs_exploitation_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 61.41 ± 16.64
  Mean Utility: 6496.99 ± 18.73
  Mean Win Rate: 0.903 ± 0.003

Player 2:
  Mean Regret: 401.85 ± 4.30
  Mean Utility: 108.30 ± 2.48
  Mean Win Rate: 0.097 ± 0.003
Summary statistics saved to: ../data/ew_vs_exploitation_summary.csv
Detailed results saved to: ../data/ew_vs_exploitation_detailed.csv
Regret history saved to: ../data/ew

In [ ]:
# Parameters for detail
n_rounds = 10000
k = 100
n_mc = 1

In [17]:
v1, v2 = 0.9, 0.3
# Use default learning rate (sqrt(log(k) / n)) - no need to specify explicitly
player1 = (ew.flexible_algorithm, v1, {'k': k, 'h': v1, 'learning_rate': None})  
player2 = (exploitation.exploitation_algorithm, v2, {'k': k, 'h': v2, 'observation_rounds': 5})
results_ew_vs_exploitation_for_detail = run_repeated_fpa(player1, player2, n_rounds, n_mc, k=k)
print("Completed: EW vs Exploitation")
plot_results(results_ew_vs_exploitation_for_detail, title="EW vs Exploitation (for detail)")

Completed: EW vs Exploitation
Plot 1 saved to: ../figures/ew_vs_exploitation_for_detail_bid_evolution.png
Plot 2 saved to: ../figures/ew_vs_exploitation_for_detail_regret.png
Plot 4 saved to: ../figures/ew_vs_exploitation_for_detail_utility_distribution.png
Plot 5 saved to: ../figures/ew_vs_exploitation_for_detail_win_rate_distribution.png

=== Summary Statistics ===
Player 1:
  Mean Regret: 67.21 ± 0.00
  Mean Utility: 6478.25 ± 0.00
  Mean Win Rate: 0.901 ± 0.000

Player 2:
  Mean Regret: 394.42 ± 0.00
  Mean Utility: 111.49 ± 0.00
  Mean Win Rate: 0.099 ± 0.000
Summary statistics saved to: ../data/ew_vs_exploitation_for_detail_summary.csv
Detailed results saved to: ../data/ew_vs_exploitation_for_detail_detailed.csv
Regret history saved to: ../data/ew_vs_exploitation_for_detail_regret_history.csv
Bid history saved to: ../data/ew_vs_exploitation_for_detail_bid_history.csv
